# 2. Pre-procesado de los datos - Detección de noticias falsas

José Luis Aguilera Luzania

## Introducción

**¿Qué es la detección de *Fake News*?**
La detección de noticias falsas (Fake News) es la tarea de evaluar la veracidad de las afirmaciones en las noticias. Este es un problema crítico en el Procesamiento del Lenguaje Natural (PLN) porque tanto en medios de noticias tradicionales como en medios digitales las Fake News generan un gran impacto social y político en cada individuo. Por ejemplo, la exposición a las Fake News puede generar actitudes de ineficacia, alienación y cinismo hacia ciertos candidatos políticos (Balmas, 2014).

**Objetivo de la libreta**
El objetivo de esta libreta es separar los datos que necesitamos de todo el conjunto de datos, limpiar los datos necesarios y estructurarlos para un análisis exploratorio.

## Librerías y los datos

**Librerías**

- Manipulación de datos:
    - `pandas`: Librería para manipular los datos de forma tabular.
    - `unicode`: Librería para eliminar acentos de las palabras.
    - `re`: Librería para utilizar expresiones regulares.

In [16]:
import pandas as pd
from utils import procesar_texto

### Leer los datos

Los datos están divididos en los archivos `train.xlsx` y `development.xlsx`, con 80% para entrenamiento y 20% para pruebas respectivamente.

Para cargar los conjuntos de datos se utilizará la librería `pandas` y su estructura `DataFrame`.

In [17]:
df_entrenamiento = pd.read_excel('Datos/train.xlsx')
df_pruebas = pd.read_excel('Datos/development.xlsx')

### Unión de los conjuntos de datos

Realizar las mismas operaciones sobre los dos conjuntos de datos y los mismos análisis puede ser contraproducente, un proceso tardado y tedioso, por lo tanto, se unirán los dos `DataFrame` en uno. Durante el proceso previo al entrenamiento los datos serán modificados y divididos de nuevo, pero eso será después del análisis exploratorio.

In [18]:
df_datos = pd.concat([df_entrenamiento, df_pruebas])
df_datos

,Id,Category,Topic,Source,Headline,Text,Link
0,1,Fake,Education,El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",http://www.elruinaversal.com/2017/06/10/rae-in...
1,2,Fake,Education,Hay noticia,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea...",https://haynoticia.es/la-palabra-haiga-aceptad...
2,3,Fake,Education,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,http://www.elruinaversal.com/2018/05/06/yordi-...
3,4,True,Education,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...
4,5,Fake,Education,Lamula,pretenden aprobar libros escolares con conteni...,Alerta: pretenden aprobar libros escolares con...,https://redaccion.lamula.pe/2018/06/19/memoria...
...,...,...,...,...,...,...,...
290,291,True,Entertainment,HUFFPOST,Meryl Streep disfrutó unos premios Oscar tan m...,Meryl Streep disfrutó unos premios Oscar tan m...,https://www.huffingtonpost.com.mx/2018/03/06/m...
291,292,Fake,Entertainment,La voz popular,EL PLAGIO DE LANA DEL REY A RADIOHEAD FUE ACOR...,EL PLAGIO DE LANA DEL REY A RADIOHEAD FUE ACOR...,http://lavozpopular.com/plagio-lana-del-rey-ra...
292,293,True,Entertainment,Billboard,Ricardo Arjona lanza una serie documental por ...,Ricardo Arjona lanza una serie documental por ...,http://www.billboard.com.ar/noticia/2980/ricar...
293,294,True,Entertainment,EL UNIVERSAL,Raúl Araiza sorprende a Andrea Legarreta con b...,Raúl Araiza sorprende a Andrea Legarreta con b...,http://www.eluniversal.com.mx/espectaculos/rau...


#### Información de los datos

In [19]:
print('Noticias verdaderas: {}'.format(df_datos.groupby('Category').size()[0]))
print('Noticias falsas: {}'    .format(df_datos.groupby('Category').size()[1]))
print(f'Todal de Noticias: {df_datos.shape[0]}')
print(f'Columnas: {list(df_datos.columns)}')

Noticias verdaderas: 480
Noticias falsas: 491
Todal de Noticias: 971
Columnas: ['Id', 'Category', 'Topic', 'Source', 'Headline', 'Text', 'Link']


### ¿Por qué pre-procesar los datos?

Durante el análisis de los datos en la libreta anterior <<1. Análisis de los datos>> se observó que varias fuentes se encontraban escritas de forma diferente, esto provoca un incremento en el número de fuentes diferentes que se obtienen y, por lo tanto, el número de fuentes obtenidas era mucho mayor al número de fuentes real, para solucionar este problema es necesario pre-procesar los datos.

Con el propósito de evitar problemas futuros como el conteo incorrecto de las fuentes, se realizara una eliminación de datos innecesarios y una limpieza de los datos a utilizar.

**¿Cuáles son los pasos a seguir?**

1. Eliminar las columnas `Id` y `Link` de los conjuntos de datos.
2. Procesar las columnas `Category`, `Topic`, `Source`, `Headline`, `Text` como se procesaron las fuentes en la libreta anterior.

#### Eliminar las columnas `Id` y `Link`

Los datos de la columna:
- `Id` son útiles para identificar cada una de las instancias de las noticias.
- `Link` son útiles para leer el artículo original o comprobar si la noticia existe.

en este caso, no son datos necesarios o útiles para el siguiente análisis.

In [20]:
# Paso 1
df_datos = df_entrenamiento.drop(columns=['Id', 'Link'])

# Paso 2
print(f'Columnas: {list(df_datos.columns)}')

Columnas: ['Category', 'Topic', 'Source', 'Headline', 'Text']


#### Procesamiento de las columnas: `Category`, `Topic`, `Source`, `Headline`, `Text`

Con el fin de agilizar el proceso de procesamiento, se definirán dos funciones: `procesar_texto` y `procesar_columnas`.

- La función `procesar_textos` es la función `procesar_fuentes` de la libreta anterior, solo que renombrada para que se ajuste a su nuevo propósito.
- La función `procesar_columnas` aplicará la función `procesar_textos` a las columnas especificadas de un `DataFrame`.

Como recordatorio la función `procesar_textos` realiza lo siguiente:
1. Convertir el texto en minúsculas.
2. Eliminar acentos.
3. Eliminar todo lo que no sea una palabra.

La implementación de la función se encuentra en el archivo `utils.py` y su definición es la siguiente:
```python
def procesar_texto(texto):
    """
    Input:
        texto: una cadena de texto.

    Output:
        texto: una cadena de texto formada solo por las palabras del texto original, sin acentos, caracteres especiales o espacios extra.
    """
    # 1. Convertir el texto en minúsculas.
    texto = texto.lower()

    # 2. Eliminar acentos.
    texto = unidecode.unidecode(texto)

    # 3. Quedarnos solo con las palabras.
    texto = re.findall(r'\w+', texto)
    texto = ' '.join(texto)

    return texto
```

Con el fin de procesar varias columnas a la vez, también se definio la función `procesar_columnas`:

In [21]:
def procesar_columnas(df, columnas, funcion_procesar):
    """
    Input:
        df: DataFrame con las columnas a procesar.
        columnas: lista de columnas a procesar.
        funcion_procesar: función que se se utilizará para procesar.

    Output: texto: una cadena de texto formada solo por las palabras del texto original, sin acentos, caracteres
    especiales o espacios extra.
    :type df: DataFrame
    """
    for col in columnas:
        df[col] = df[col].apply(funcion_procesar)
    return df

In [22]:
# Lista de columnas a procesar.
columnas_a_procesar = ['Category', 'Topic', 'Source', 'Headline', 'Text']

# Procesamiento.
df_datos = procesar_columnas(df_datos, columnas_a_procesar, procesar_texto)

### Renombrar las columnas de los DataFrame.

Las columnas se encuentran escritas en inglés, pero los datos como el texto en español, para evitar inconsistencias en el análisis exploratorio las columnas se traducirán al español.

In [23]:
columnas_dict = {
    'Category': 'Categoria',
    'Topic':    'Tema',
    'Source':   'Fuente',
    'Headline': 'Encabezado',
    'Text':     'Texto'
}

# Renombrar las columnas.
df_datos.rename(columns=columnas_dict, inplace=True)

# Imprimir las columnas.
print(f'Columnas: {list(df_datos.columns)}')

Columnas: ['Categoria', 'Tema', 'Fuente', 'Encabezado', 'Texto']


### Renombrar los temas
Los temas también deberían estar en español.

Como recordatorio los temas abarcados en las noticias son los siguientes:
1. Ciencia.
2. Deportes.
3. Economía.
4. Entretenimiento.
5. Educación.
6. Política.
7. Salud.
8. Seguridad.
9. Sociedad

In [24]:
def renombrar_temas(df):
    df.loc[df['Tema'] == 'economy'      , 'Tema'] = 'economía'
    df.loc[df['Tema'] == 'education'    , 'Tema'] = 'educación'
    df.loc[df['Tema'] == 'entertainment', 'Tema'] = 'entretenimiento'
    df.loc[df['Tema'] == 'health'       , 'Tema'] = 'salud'
    df.loc[df['Tema'] == 'politics'     , 'Tema'] = 'política'
    df.loc[df['Tema'] == 'science'      , 'Tema'] = 'ciencia'
    df.loc[df['Tema'] == 'security'     , 'Tema'] = 'seguridad'
    df.loc[df['Tema'] == 'society'      , 'Tema'] = 'sociedad'
    df.loc[df['Tema'] == 'sport'        , 'Tema'] = 'deportes'

renombrar_temas(df_datos)

### Renombrar las categorías
Las categorías también serán traducidas al español.

In [25]:
def renombrar_categorias(df):
    df.loc[df['Categoria'] == 'fake', 'Categoria'] = 'falsa'
    df.loc[df['Categoria'] == 'true', 'Categoria'] = 'verdadera'

renombrar_categorias(df_datos)

## Datos procesados

In [26]:
df_datos.head()

,Categoria,Tema,Fuente,Encabezado,Texto
0,falsa,educación,el ruinaversal,rae incluira la palabra lady en el diccionario...,rae incluira la palabra lady en el diccionario...
1,falsa,educación,hay noticia,la palabra haiga aceptada por la rae,la palabra haiga aceptada por la rae la real a...
2,falsa,educación,el ruinaversal,yordi rosado escribira y disenara los nuevos l...,yordi rosado escribira y disenara los nuevos l...
3,verdadera,educación,el universal,unam capacitara a maestros para aprobar prueba...,unam capacitara a maestros para aprobar prueba...
4,falsa,educación,lamula,pretenden aprobar libros escolares con conteni...,alerta pretenden aprobar libros escolares con ...


## Guardar los DataFrame en archivos de texto.

Ahora los datos se encuentran procesados y totalmente en español, es necesario guardarlos en un archivo para poder utilizarlo en las libretas posteriores. Los datos se guardarán en el archivo con extensión `.csv`.

In [27]:
df_datos.to_csv('Datos/datos.csv', index=False)